# Understand the Difference Between Return Sequences and Return States for LSTMs in Keras

The Keras API provides access to both return sequences and return state. The use and difference between these data can be confusing when designing sophisticated recurrent neural network models, such as the encoder-decoder model.

Summary:
* That return sequences return the hidden state output for each input time step.
* That return state returns the hidden state output and cell state for the last input time step.
* That return sequences and return state can be used at the same time.

The Long Short-Term Memory, or LSTM, is a recurrent neural network that is comprised of internal gates.

Unlike other recurrent neural networks, the network’s internal gates allow the model to be trained successfully using backpropagation through time, or BPTT, and avoid the vanishing gradients problem.

In the Keras deep learning library, LSTM layers can be created using the LSTM() class.

Creating a layer of LSTM memory units allows you to specify the number of memory units within the layer.

Each unit or cell within the layer has an internal cell state, often abbreviated as *“c“*, and outputs a hidden state, often abbreviated as *“h“*.

![](https://imgur.com/nXBgbtq.png)

The Keras API allows you to access these data, which can be useful or even required when developing sophisticated recurrent neural network architectures, such as the encoder-decoder model.


In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.2'

## Return Sequences
Each LSTM cell will output one hidden state *h* for each input.
> h = LSTM(X)

We can demonstrate this in Keras with a very small model with a single LSTM layer that itself contains a single LSTM cell.
In this example, we will have one input sample with 3 time steps and one feature observed at each time step:
> timestep_1 = 0.1

> timestep_2 = 0.2

> timestep_3 = 0.3

In [2]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
import numpy as np

# define model
inputs1 = Input(shape=(3, 1))
lstm1 = LSTM(1)(inputs1)
model = Model(inputs=inputs1, outputs=lstm1)

# define input data
# The input tensor format required by the LSTM model is:
# (batch_size，timesteps，input_features)
data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))

# make and show prediction
print(model.predict(data))

[[-0.06489331]]


Running the example outputs a single hidden state for the input sequence with 3 time steps.

It is possible to access the hidden state output for each input time step.

This can be done by setting the `return_sequences` attribute to `True` when defining the LSTM layer, as follows:
> LSTM(1, return_sequences=True)

In [3]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
import numpy as np

# define model
inputs1 = Input(shape=(3, 1))
lstm1 = LSTM(1, return_sequences=True)(inputs1)
model = Model(inputs=inputs1, outputs=lstm1)

# define input data
data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))

# make and show prediction
print(model.predict(data))

[[[-0.01621807]
  [-0.04534798]
  [-0.08408501]]]


Running the example returns a sequence of 3 values, one hidden state output for each input time step for the single LSTM cell in the layer.

You must set `return_sequences=True` when stacking LSTM layers so that the second LSTM layer has a three-dimensional sequence input. For more details, see the post:
* [Stacked Long Short-Term Memory Networks](https://machinelearningmastery.com/stacked-long-short-term-memory-networks/)

You may also need to access the sequence of hidden state outputs when predicting a sequence of outputs with a Dense output layer wrapped in a TimeDistributed layer. See this post for more details:

* [How to Use the TimeDistributed Layer for Long Short-Term Memory Networks in Python](https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/)

## Return States
The output of an LSTM cell or layer of cells is called the hidden state.

This is confusing, because each LSTM cell retains an internal state that is not output, called the cell state, or *c*.

Generally, we do not need to access the cell state unless we are developing sophisticated models where subsequent layers may need to have their cell state initialized with the final cell state of another layer, such as in an encoder-decoder model.

Keras provides the `return_state` argument to the LSTM layer that will provide access to the hidden state output (`state_h`) and the cell state (`state_c`). For example:
> lstm1, state_h, state_c = LSTM(1, return_state=True)

This may look confusing because both `lstm1` and `state_h` refer to the same hidden state output. The reason for these two tensors being separate will become clear in the next section.

We can demonstrate access to the hidden and cell states of the cells in the LSTM layer with a worked example listed below.

In [4]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
import numpy as np

# define model
inputs1 = Input(shape=(3, 1))
lstm1, state_h, state_c = LSTM(1, return_state=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])

# define input data
data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))

# make and show prediction
print(model.predict(data))

[array([[ 0.05934551]], dtype=float32), array([[ 0.05934551]], dtype=float32), array([[ 0.13478732]], dtype=float32)]


Running the example returns 3 arrays:

1. The LSTM hidden state output for the last time step.
2. The LSTM hidden state output for the last time step (again).
3. The LSTM cell state for the last time step.

The hidden state and the cell state could in turn be used to initialize the states of another LSTM layer with the same number of cells.

## Return States and Sequences
We can access both the sequence of hidden state and the cell states at the same time.

This can be done by configuring the LSTM layer to both return sequences and return states.
> lstm_1, state_h, state_c = LSTM(1, return_sequences=True, return_state=True)



In [5]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
import numpy as np

# define model
inputs1 = Input(shape=(3, 1))
lstm1, state_h, state_c = LSTM(1, return_sequences=True, return_state=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])

# define input data
data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))

# make and show prediction
print(model.predict(data))

[array([[[ 0.02139024],
        [ 0.06493985],
        [ 0.13029268]]], dtype=float32), array([[ 0.13029268]], dtype=float32), array([[ 0.23715121]], dtype=float32)]


Running the example, we can see now why the LSTM output tensor and hidden state output tensor are declared separably.

The layer returns the hidden state for each input time step, then separately, the hidden state output for the last time step and the cell state for the last input time step.

This can be confirmed by seeing that the last value in the returned sequences (first array) matches the value in the hidden state (second array).

# Reference 
* [Understand the Difference Between Return Sequences and Return States for LSTMs in Keras](https://machinelearningmastery.com/return-sequences-and-return-states-for-lstms-in-keras/)
* [A ten-minute introduction to sequence-to-sequence learning in Keras](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)